# Fiddler Environment Statistics

This notebook extracts hierarchical information about projects, models, and features from a Fiddler environment

In [ ]:
# Setup: Import utilities and initialize connection
from fiddler_utils import get_or_init, EnvironmentReporter

In [ ]:
# Configuration
URL = ''  # Example: 'https://your_company_name.fiddler.ai'
TOKEN = ''  # Get this from the Settings > Credentials tab in Fiddler UI
EXPORT_CSV = False
TOP_N = 15  # Number of items to show in top lists

In [ ]:
# Initialize connection
get_or_init(url=URL, token=TOKEN, log_level='ERROR')

# Create reporter
reporter = EnvironmentReporter()

print("✓ Successfully connected to Fiddler")
print("✓ EnvironmentReporter initialized")

In [ ]:
# Run environment analysis
# This traverses all projects, models, and features
reporter.analyze_environment(
    include_features=True,
    include_timestamps=True,
    include_assets=False  # Set to True if you need segment/metric/alert counts (slower)
)

print("✓ Environment analysis complete")

In [ ]:
# Generate and display comprehensive report
reporter.generate_report(
    show_projects=True,
    show_models=True,
    show_timestamps=True,
    show_newest_oldest=True,
    top_n=TOP_N
)

In [ ]:
# Export to CSV files
if EXPORT_CSV:
    files = reporter.export_to_csv(
        output_dir='.',
        prefix='env_stats'
    )
    print(f"\n✓ Exported {len(files)} CSV files:")
    for file_path in files:
        print(f"  • {file_path}")
else:
    print("\nCSV export skipped (EXPORT_CSV = False)")

In [ ]:
# Optional: Access raw data for custom analysis
# You can access the underlying data structures for custom processing

# Get statistics object
stats = reporter.get_statistics()
print("Statistics available:")
print(f"  • Total projects: {stats.total_projects}")
print(f"  • Total models: {stats.total_models}")
print(f"  • Total features: {stats.total_features}")
print(f"  • Average models per project: {stats.models_per_project_mean:.1f}")
print(f"  • Average features per model: {stats.features_per_model_mean:.1f}")

# Get hierarchy for detailed inspection
hierarchy = reporter.get_hierarchy()
print(f"\nHierarchy contains {len(hierarchy.projects)} projects")

# Example: Find projects with most models
if stats.top_projects_by_models:
    top_project_name, top_project_count = stats.top_projects_by_models[0]
    print(f"\nTop project: '{top_project_name}' with {top_project_count} models")

# Get timestamp analysis if available
ts_analysis = reporter.get_timestamp_analysis()
if ts_analysis:
    print(f"\nTimestamp coverage: {ts_analysis.timestamp_coverage_pct:.1f}%")